In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

df = pd.read_csv('/home/user/Документы/Projects/Python/pml/assessment/wdbc.data')
X = df.iloc[:, 2:].values
y = df.iloc[:, 1].values
le = LabelEncoder()
y = le.fit_transform(y)
le.classes_

array(['B', 'M'], dtype=object)

In [2]:
le.transform(['M', 'B'])

array([1, 0])

In [3]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=1)

In [4]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline

pipe_lr = make_pipeline(StandardScaler(),
                        PCA(n_components=2),
                        LogisticRegression(random_state=1, solver='lbfgs'))
pipe_lr.fit(X_train, y_train)
y_pred = pipe_lr.predict(X_test)
print(f'Правильность при испытании: {pipe_lr.score(X_test, y_test)}')

Правильность при испытании: 0.9298245614035088


In [7]:
import numpy as np
from sklearn.model_selection import StratifiedKFold

kfold = StratifiedKFold(n_splits=10).split(X_train, y_train)
scores = []
for k, (train, test) in enumerate(kfold):
    pipe_lr.fit(X_train[train], y_train[train])
    score = pipe_lr.score(X_train[test], y_train[test])
    scores.append(score)
    print('Блок: %2d, Распределение классов: %s, Правильность: %.3f' % (k + 1, np.bincount(y_train[train]), score))

Блок:  1, Распределение классов: [256 152], Правильность: 0.913
Блок:  2, Распределение классов: [256 152], Правильность: 0.935
Блок:  3, Распределение классов: [256 152], Правильность: 0.957
Блок:  4, Распределение классов: [256 152], Правильность: 0.891
Блок:  5, Распределение классов: [257 152], Правильность: 0.978
Блок:  6, Распределение классов: [257 152], Правильность: 0.978
Блок:  7, Распределение классов: [257 152], Правильность: 0.978
Блок:  8, Распределение классов: [257 152], Правильность: 0.911
Блок:  9, Распределение классов: [257 152], Правильность: 0.933
Блок: 10, Распределение классов: [256 153], Правильность: 0.978


In [11]:
print('\nТочность перекрестной проверки: %.3f +/- %.3f' % (np.mean(scores), np.std(scores)))


Точность перекрестной проверки: 0.945 +/- 0.031


In [12]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(estimator=pipe_lr, X=X_train, y=y_train, cv=10, n_jobs=1)
print('Меры правильности перекрестной проверки: %s' % scores)

Меры правильности перекрестной проверки: [0.91304348 0.93478261 0.95652174 0.89130435 0.97777778 0.97777778
 0.97777778 0.91111111 0.93333333 0.97777778]
